In [ ]:
import pandas as pd
import numpy as np
df_datacenters = pd.read_csv("datacenters_imputed_df.csv")
df_green = pd.read_csv("imputed_df.csv")

#Creating a new elite country level df for supercomputers with new columns added in (numerical).
green500_country = df_green.groupby('Country').agg({
    'Energy Efficiency [GFlops/Watts]': ['mean', 'median', 'max'], #subsetted country to include their average energy eficiency, thier median and the max. 
    'Power (kW)': ['sum', 'mean'],
    'Total Cores': ['sum', 'mean'],
    'Rmax [TFlop/s]': ['sum', 'mean'],  
    'Year': ['max', 'min'], 
    'TOP500 Rank': 'count',
    'Accelerator/Co-Processor Cores': 'mean'
}).reset_index()

green500_country.columns = ['_'.join(col).strip('_') for col in green500_country.columns.values]
green500_country.rename(columns={'Country': 'country'}, inplace=True)

# Renaming columns
green500_country.columns = [
    'country',
    'avg_energy_efficiency', 'median_energy_efficiency', 'max_energy_efficiency',
    'total_power_kw', 'avg_power_kw',
    'total_cores', 'avg_cores',
    'total_performance_tflops', 'avg_performance_tflops',
    'most_recent_year', 'oldest_year',
    'num_supercomputers',
    'avg_accelerator_cores'
]

In [ ]:
# Left join to keep ALL countries from datacenters dataset
merged_df = pd.merge(
    df_datacenters,
    green500_country,
    on='country',
    how='left',
    indicator=True
)

# Create categorical variable
merged_df['supercomputing_category'] = merged_df['_merge'].map({
    'both': 'Elite Supercomputing Nation',
    'left_only': 'Infrastructure Only'
})

# Fill NaN for countries without supercomputers
merged_df['num_supercomputers'] = merged_df['num_supercomputers'].fillna(0)

# Save merged dataset
merged_df.to_csv("merged_infrastructure_efficiency.csv", index=False)

print(f"Total countries: {len(merged_df)}")
print(f"Elite nations (in Green500): {(merged_df['_merge'] == 'both').sum()}")
print(f"Infrastructure only: {(merged_df['_merge'] == 'left_only').sum()}")

Total countries: 191
Elite nations (in Green500): 29
Infrastructure only: 162


In [3]:
print(merged_df['supercomputing_category'].value_counts())
print(merged_df[merged_df['supercomputing_category'] == 'Elite Supercomputing Nation']['country'].tolist())


supercomputing_category
Infrastructure Only            162
Elite Supercomputing Nation     29
Name: count, dtype: int64
['United States', 'Germany', 'United Kingdom', 'China', 'France', 'Canada', 'Australia', 'Netherlands', 'Japan', 'Brazil', 'India', 'Sweden', 'Spain', 'Poland', 'Switzerland', 'Austria', 'Belgium', 'Czechia', 'Denmark', 'Finland', 'Hungary', 'Italy', 'Luxembourg', 'Morocco', 'Norway', 'Portugal', 'Saudi Arabia', 'Slovakia', 'United Arab Emirates']


In [1]:
import pandas as pd
df = pd.read_csv("merged_infrastructure_efficiency.csv")
df.columns.to_list()

['country',
 'cooling_technologies_common',
 'total_data_centers',
 'hyperscale_data_centers',
 'colocation_data_centers',
 'floor_space_sqft_total',
 'power_capacity_MW_total',
 'average_renewable_energy_usage_percent',
 'internet_penetration_percent',
 'growth_rate_of_data_centers_percent_per_year',
 'region',
 'avg_energy_efficiency',
 'median_energy_efficiency',
 'max_energy_efficiency',
 'total_power_kw',
 'avg_power_kw',
 'total_cores',
 'avg_cores',
 'total_performance_tflops',
 'avg_performance_tflops',
 'most_recent_year',
 'oldest_year',
 'num_supercomputers',
 'avg_accelerator_cores',
 '_merge',
 'supercomputing_category']